In [ ]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np

experiment_name = "N50000_t500_b100_lr1_5_D1" # "N50000_t500_b100_lr1_5"
bellman_iterations_scopes = [1, 4, 7, 10, 20]
seeds = [int(f"1{idx}") for idx in range(1, 21)]

p = json.load(open(f"figures/{experiment_name}/parameters.json"))

In [ ]:
import matplotlib.pyplot as plt 

from experiments.car_on_hill import COLORS


plt.rc("font", size=15)
plt.rc("lines", linewidth=3)

table_indexes = [1, 6, 11, 16, 20]
table = {"K": []}
for letter in ["Q", "w"]:
    for idx in table_indexes:
        table[f"i = {idx}, {letter}"] = []

for idx_bellman_iterations_scope, bellman_iterations_scope in enumerate(bellman_iterations_scopes):
    diff_weights_seeds = []
    for seed in seeds:
        diff_weights_seeds.append(np.load(f"figures/{experiment_name}/iFQI_linear/{bellman_iterations_scope}_distance_w_s{seed}.npy"))

    diff_weights = np.array(diff_weights_seeds).mean(axis=0)
    # high_weights = np.array(diff_weights_seeds).max(axis=0)
    # low_weights = np.array(diff_weights_seeds).min(axis=0)

    frozen_line = p["n_bellman_iterations"] - bellman_iterations_scope + 1
    plt.plot(range(1, frozen_line + 1), diff_weights[1: frozen_line + 1], label=f"K={bellman_iterations_scope}", c=COLORS[str(bellman_iterations_scope)])
    plt.plot(range(frozen_line, p["n_bellman_iterations"] + 1), diff_weights[frozen_line:], linestyle="dotted", c=COLORS[str(bellman_iterations_scope)])
    # plt.fill_between(range(1, p["n_bellman_iterations"] + 1), low_weights[1:], high_weights[1:], alpha=0.3)

    table["K"].append(bellman_iterations_scope)
    [table[f"i = {idx}, w"].append(diff_weights[idx]) for idx in table_indexes]



plt.xticks(range(1, p["n_bellman_iterations"] + 1, 3))
plt.xlabel("#Iteration k")
plt.ylabel(r"$|| \overline{\theta}_k - \theta_k ||_2$")
plt.title("Car-On-Hill")
plt.legend().set_zorder(3)
plt.grid(zorder=0)
# _ = plt.savefig(f"figures/{experiment_name}/distance_to_optimal_Q.pdf", bbox_inches='tight')

In [ ]:
for idx_bellman_iterations_scope, bellman_iterations_scope in enumerate(bellman_iterations_scopes):
    diff_q_values_seeds = []
    for seed in seeds:
        diff_q_values_seeds.append(np.load(f"figures/{experiment_name}/iFQI_linear/{bellman_iterations_scope}_distance_Q_s{seed}.npy"))

    diff_q_values = np.array(diff_q_values_seeds).mean(axis=0)

    frozen_line = p["n_bellman_iterations"] - bellman_iterations_scope + 1
    plt.plot(range(1, frozen_line + 1), diff_q_values[1: frozen_line + 1], label=f"K={bellman_iterations_scope}", c=COLORS[str(bellman_iterations_scope)])
    plt.plot(range(frozen_line, p["n_bellman_iterations"] + 1), diff_q_values[frozen_line:], linestyle="dotted", c=COLORS[str(bellman_iterations_scope)])

    [table[f"i = {idx}, Q"].append(diff_q_values[idx]) for idx in table_indexes]

plt.xticks(range(1, p["n_bellman_iterations"] + 1, 3))
plt.xlabel("#Iteration k")
plt.ylabel(r"$|| Q^* - Q_{\theta_k}||_2$")
plt.title("Car-On-Hill")
plt.legend().set_zorder(3)
plt.grid(zorder=0)
# _ = plt.savefig(f"figures/{experiment_name}/distance_to_optimal_Q.pdf", bbox_inches='tight')

In [ ]:
import pandas as pd 

pd.DataFrame(table).round(2).T